# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'12-21-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'12-21-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-12-22 05:22:33,33.93911,67.709953,50013,2082,39508,8423.0,Afghanistan,128.474522,4.162918
1,NaN,NaN,NaN,Albania,2020-12-22 05:22:33,41.15330,20.168300,53425,1098,28752,23575.0,Albania,1856.452846,2.055218
2,NaN,NaN,NaN,Algeria,2020-12-22 05:22:33,28.03390,1.659600,95659,2675,63644,29340.0,Algeria,218.145324,2.796391
3,NaN,NaN,NaN,Andorra,2020-12-22 05:22:33,42.50630,1.521800,7602,81,7028,493.0,Andorra,9838.866240,1.065509
4,NaN,NaN,NaN,Angola,2020-12-22 05:22:33,-11.20270,17.873900,16686,390,9695,6601.0,Angola,50.769379,2.337289


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,48229,48527,48718,48952,49161,49378,49621,49681,49817,50013
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,47742,48530,49191,50000,50637,51424,52004,52542,53003,53425
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,91638,92102,92597,93065,93507,93933,94371,94781,95203,95659


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1956,1965,1969,1995,2011,2025,2030,2047,2067,2082
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,989,1003,1016,1028,1040,1055,1066,1074,1088,1098
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2584,2596,2609,2623,2631,2640,2647,2659,2666,2675


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,38200,38250,38252,38336,38475,38505,38540,38613,39006,39508
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,24520,24820,24820,25876,26381,26898,27426,27831,28121,28752
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,60028,60457,60888,61307,61700,62089,62487,62869,63260,63644


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,3258,3300,3329,3426,3510,3570,3647,3698,3741,3780
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,10665,10806,10898,11061,11212,11364,11556,11722,11827,11952
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1269,1272,1275,1292,1296,1309,1318,1330,1336,1336


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,41,41,41,43,43,43,44,44,44,44
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,141,141,141,141,145,145,146,147,147,147
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,30,30,30,30,30,30,30,32,32,32


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
649,1001.0,Autauga,Alabama,US,2020-12-22 05:22:33,32.539527,-86.644082,3780,44,0,3736.0,"Autauga, Alabama, US",6765.827203,1.164021
686,1075.0,Lamar,Alabama,US,2020-12-22 05:22:33,33.779950,-88.096680,966,12,0,954.0,"Lamar, Alabama, US",6997.464687,1.242236
687,1077.0,Lauderdale,Alabama,US,2020-12-22 05:22:33,34.901719,-87.656247,5644,64,0,5580.0,"Lauderdale, Alabama, US",6086.553290,1.133948


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,18035209,319364,0
India,10075116,146111,9636487
Brazil,7263619,187291,6469310


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,18035209,319364,0,17715845,2020-12-22 05:22:33,37.936303,-91.379001
India,10075116,146111,9636487,292518,2020-12-22 05:22:33,23.088275,81.806127
Brazil,7263619,187291,6469310,607018,2020-12-22 05:22:33,-12.669522,-48.480493
Russia,2850042,50723,2273510,525809,2020-12-22 05:22:33,54.546312,62.120860
France,2535716,61019,190296,2284401,2020-12-22 05:22:33,8.314863,-21.695174


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,1923887,22837,0
Texas,1604991,25936,0
Florida,1212581,20680,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,1923887,22837,0,1901050,2020-12-22 05:22:33,37.843962,-120.728594
Texas,1604991,25936,0,1579055,2020-12-22 05:22:33,31.660643,-98.653069
Florida,1212581,20680,0,1191901,2020-12-22 05:22:33,28.940755,-82.700744
Illinois,905069,16527,0,888542,2020-12-22 05:22:33,39.843618,-89.178330
New York,857049,36568,0,820481,2020-12-22 05:22:33,42.544151,-75.474183


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,634849,8931,0
Illinois,Cook,373008,7802,0
Arizona,Maricopa,284462,4685,0
Florida,Miami-Dade,276414,4097,0
Texas,Harris,221860,3245,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,634849,8931,0,625918,2020-12-22 05:22:33,34.308284,-118.228241,6037.0
Illinois,Cook,373008,7802,0,365206,2020-12-22 05:22:33,41.841448,-87.816588,17031.0
Arizona,Maricopa,284462,4685,0,279777,2020-12-22 05:22:33,33.348359,-112.491815,4013.0
Florida,Miami-Dade,276414,4097,0,272317,2020-12-22 05:22:33,25.611236,-80.551706,12086.0
Texas,Harris,221860,3245,0,218615,2020-12-22 05:22:33,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-19,49681,52542,94781,7560,16626,152,1537169,153173,28168,337209,...,2010077,12557,75806,1,109781,1411,121216,2087,18620,12151
2020-12-20,49817,53003,95203,7577,16644,153,1541285,153825,28198,338854,...,2046161,13048,75933,1,110075,1413,122643,2087,18716,12325
2020-12-21,50013,53425,95659,7602,16686,153,1547138,154065,28219,340373,...,2079678,13477,76048,1,110513,1414,123945,2087,18768,12422


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-19,2047,1074,2659,80,386,5,41763,2616,908,5209,...,67177,114,612,0,984,35,1117,606,373,318
2020-12-20,2067,1088,2666,81,387,5,41813,2630,908,5351,...,67503,119,612,0,988,35,1141,606,373,320
2020-12-21,2082,1098,2675,81,390,5,41997,2656,908,5435,...,67718,119,612,0,993,35,1169,606,375,322


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-19,38613,27831,62869,6963,9518,141,1362617,131931,25704,300611,...,4354,8002,73099,1,104208,1269,96461,1384,17756,9984
2020-12-20,39006,28121,63260,6997,9592,143,1368346,132532,25709,303213,...,4375,8327,73242,1,104462,1269,97979,1384,17762,10024
2020-12-21,39508,28752,63644,7028,9695,143,1374401,133176,25716,305658,...,4386,8688,73339,1,104799,1269,99547,1384,17818,10112


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,3258,3300,3329,3426,3510,3570,3647,3698,3741,3780
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,841,855,858,870,889,912,925,941,946,966
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,5005,5078,5125,5200,5243,5346,5428,5531,5582,5644
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,1569,1585,1596,1617,1635,1672,1708,1739,1757,1783
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,8675,8744,8788,8857,8966,9055,9188,9283,9337,9372


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-19,319904,43688,0,448231,200114,1842557,306598,162782,50136,49,...,503651,1584207,248970,6343,1910,302972,222600,71215,490679,41488
2020-12-20,322452,43871,0,453597,201650,1884033,308890,162782,50616,49,...,519687,1593471,250964,6443,1921,306848,222600,72342,492724,41662
2020-12-21,324832,44023,0,461345,203107,1923887,311036,167377,51056,49,...,529578,1604991,252783,6534,1921,310890,226635,73337,494353,42118


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2020-12-19        3698   11722    1330  1601   4191     752   1393    8594   
2020-12-20        3741   11827    1336  1613   4218     753   1399    8648   
2020-12-21        3780   11952    1336  1628   4234     754   1405    8684   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-12-19         2116     1278  ...    1771    407     2394      563   
2020-12-20         2125     1287  ...    1784    410     2401      565   
2020-12-21         2133     1294  ...    1812    412     2420      567   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-12-19           2702  1992  1406          0      695    443  
2020-12-20           2718  2002  1416          0      699    446  
2020-12-21           2762  2019  1446          0      713    447  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-19,4389,184,0,7937,3191,22582,4339,5581,858,0,...,5960,25714,1148,108,23,4643,3104,1122,4722,351
2020-12-20,4389,184,0,7971,3237,22675,4368,5581,862,0,...,6071,25827,1155,111,23,4650,3104,1128,4743,351
2020-12-21,4389,184,0,7972,3295,22837,4378,5676,871,0,...,6136,25936,1161,111,23,4654,3106,1129,4751,351


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-12-19          44     147      32   42     54      20     45     140   
2020-12-20          44     147      32   42     54      20     45     140   
2020-12-21          44     147      32   42     54      20     45     140   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-12-19           60       23  ...       7      8       15        5   
2020-12-20           60       23  ...       7      8       15        5   
2020-12-21           60       23  ...       7      8       15        5   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-12-19             15     2     7          0       11      2  
2020-12-20             15     2     7          0       11      2  
2020-12-21             15     2     7          0       11      2  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-19,0.001209,0.010345,0.004345,0.005453,0.003864,0.000000,0.003784,0.006043,0.001493,0.006856,...,0.013742,0.050795,0.001731,0.0,0.003528,0.000709,0.013410,0.0,0.002423,0.008633
2020-12-20,0.002737,0.008774,0.004452,0.002249,0.001083,0.006579,0.002678,0.004257,0.001065,0.004878,...,0.017952,0.039102,0.001675,0.0,0.002678,0.001417,0.011772,0.0,0.005156,0.014320
2020-12-21,0.003934,0.007962,0.004790,0.003299,0.002523,0.000000,0.003797,0.001560,0.000745,0.004483,...,0.016380,0.032879,0.001514,0.0,0.003979,0.000708,0.010616,0.0,0.002778,0.007870


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-19,0.008374,0.007505,0.004533,0.0000,0.005208,0.0,0.002184,0.007704,0.0,0.015994,...,0.008058,0.045872,0.0,NaN,0.005107,0.0,0.018232,0.0,0.000000,0.006329
2020-12-20,0.009770,0.013035,0.002633,0.0125,0.002591,0.0,0.001197,0.005352,0.0,0.027261,...,0.004853,0.043860,0.0,NaN,0.004065,0.0,0.021486,0.0,0.000000,0.006289
2020-12-21,0.007257,0.009191,0.003376,0.0000,0.007752,0.0,0.004401,0.009886,0.0,0.015698,...,0.003185,0.000000,0.0,NaN,0.005061,0.0,0.024540,0.0,0.005362,0.006250


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-19,0.001894,0.014767,0.006113,0.006359,0.018513,0.000000,0.004321,0.005472,0.000156,0.010715,...,0.021107,0.039356,0.001891,0.0,0.002241,0.00237,0.020147,0.0,0.001523,0.009096
2020-12-20,0.010178,0.010420,0.006219,0.004883,0.007775,0.014184,0.004204,0.004555,0.000195,0.008656,...,0.004823,0.040615,0.001956,0.0,0.002437,0.00000,0.015737,0.0,0.000338,0.004006
2020-12-21,0.012870,0.022439,0.006070,0.004430,0.010738,0.000000,0.004425,0.004859,0.000272,0.008064,...,0.002514,0.043353,0.001324,0.0,0.003226,0.00000,0.016003,0.0,0.003153,0.008779


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-19,0.013371,0.007751,NaN,0.012560,0.013641,0.022396,0.008191,0.000000,0.020913,0.0,...,0.000000,0.009672,0.009766,0.016018,0.005263,0.011971,0.010587,0.020989,0.008872,0.003119
2020-12-20,0.007965,0.004189,NaN,0.011972,0.007676,0.022510,0.007476,0.000000,0.009574,0.0,...,0.031840,0.005848,0.008009,0.015765,0.005759,0.012793,0.000000,0.015825,0.004168,0.004194
2020-12-21,0.007381,0.003465,NaN,0.017081,0.007225,0.021154,0.006947,0.028228,0.008693,0.0,...,0.019033,0.007230,0.007248,0.014124,0.000000,0.013173,0.018127,0.013754,0.003306,0.010945


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-12-19      0.013984  0.014365  0.009105  0.015219  0.017727  0.006693   
2020-12-20      0.011628  0.008958  0.004511  0.007495  0.006442  0.001330   
2020-12-21      0.010425  0.010569  0.000000  0.009299  0.003793  0.001328   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-12-19      0.006503  0.015959  0.012440  0.010277  ...  0.005108   
2020-12-20      0.004307  0.006283  0.004253  0.007042  ...  0.007340   
2020-12-21      0.004289  0.004163  0.003765  0.005439  ...  0.015695   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-12-19      0.002463  0.001674  0.001779   0.002226  0.001005  0.005004   
2020-12-20      0.007371  0.002924  0.003552   0.005922  0.005020  0.007112   
2020-12-21      0.004878  0.007913  0.003540   0.016188  0.008492  0.021186   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-12-19            NaN  0.007246  0.000000  
2020-12-20            NaN  0.005755  0.006772  
2020-12-21            NaN  0.020029  0.002242  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-19,0.021648,0.005464,NaN,0.015091,0.016566,0.009477,0.018784,0.000000,0.004684,NaN,...,0.000000,0.007523,0.007018,0.009346,0.0,0.009787,0.000000,0.028414,0.019430,0.0
2020-12-20,0.000000,0.000000,NaN,0.004284,0.014416,0.004118,0.006684,0.000000,0.004662,NaN,...,0.018624,0.004394,0.006098,0.027778,0.0,0.001508,0.000000,0.005348,0.004447,0.0
2020-12-21,0.000000,0.000000,NaN,0.000125,0.017918,0.007144,0.002289,0.017022,0.010441,NaN,...,0.010707,0.004220,0.005195,0.000000,0.0,0.000860,0.000644,0.000887,0.001687,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                    \
Admin2         Autauga   Baldwin   Barbour Bibb    Blount Bullock Butler   
2020-12-19         0.0  0.006849  0.066667  0.0  0.038462     0.0    0.0   
2020-12-20         0.0  0.000000  0.000000  0.0  0.000000     0.0    0.0   
2020-12-21         0.0  0.000000  0.000000  0.0  0.000000     0.0    0.0   

Province_State                               ... Wyoming                  \
Admin2           Calhoun  Chambers Cherokee  ...    Park Platte Sheridan   
2020-12-19      0.021898  0.034483      0.0  ...     0.0    0.0      0.0   
2020-12-20      0.000000  0.000000      0.0  ...     0.0    0.0      0.0   
2020-12-21      0.000000  0.000000      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-12-19          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2020-12-20          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2020-12-21          0.0        0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-19,0.002933,0.011730,0.004533,0.005576,0.004189,0.002964,0.004175,0.006119,0.001184,0.006960,...,0.014325,0.048205,0.001683,0.0,0.003268,0.001226,0.015265,0.000119,0.002862,0.011281
2020-12-20,0.002835,0.010252,0.004492,0.003912,0.002636,0.004772,0.003426,0.005188,0.001125,0.005919,...,0.016138,0.043654,0.001679,0.0,0.002973,0.001322,0.013519,0.000060,0.004009,0.012800
2020-12-21,0.003385,0.009107,0.004641,0.003606,0.002580,0.002386,0.003612,0.003374,0.000935,0.005201,...,0.016259,0.038266,0.001597,0.0,0.003476,0.001015,0.012067,0.000030,0.003394,0.010335


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-19,0.006703,0.009676,0.003861,0.003265,0.006270,0.001953,0.002949,0.007772,5.257301e-10,0.023886,...,0.007863,0.040364,3.203474e-06,NaN,0.004893,1.225516e-34,0.020053,-0.000013,0.003003,0.005998
2020-12-20,0.008237,0.011356,0.003247,0.007883,0.004430,0.000977,0.002073,0.006562,2.628650e-10,0.025573,...,0.006358,0.042112,1.601737e-06,NaN,0.004479,6.127582e-35,0.020769,-0.000007,0.001501,0.006144
2020-12-21,0.007747,0.010274,0.003311,0.003941,0.006091,0.000488,0.003237,0.008224,1.314325e-10,0.020636,...,0.004771,0.021056,8.008686e-07,NaN,0.004770,3.063791e-35,0.022655,-0.000003,0.003432,0.006197


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-19,0.001591,0.017535,0.006295,0.007107,0.014180,0.001374,0.003754,0.007545,0.000128,0.010894,...,0.020749,0.038962,0.001835,0.0,0.002339,0.003131,0.019921,-0.000014,0.002491,0.008864
2020-12-20,0.005884,0.013978,0.006257,0.005995,0.010977,0.007779,0.003979,0.006050,0.000161,0.009775,...,0.012786,0.039789,0.001896,0.0,0.002388,0.001566,0.017829,-0.000007,0.001414,0.006435
2020-12-21,0.009377,0.018208,0.006164,0.005213,0.010858,0.003890,0.004202,0.005455,0.000217,0.008919,...,0.007650,0.041571,0.001610,0.0,0.002807,0.000783,0.016916,-0.000004,0.002284,0.007607


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-19,0.014451,0.009430,NaN,0.014091,0.013908,0.025076,0.010028,0.008113,0.016506,7.009440e-85,...,0.010333,0.010599,0.010590,0.016611,0.009579,0.012000,0.011109,0.020494,0.008432,0.008083
2020-12-20,0.011208,0.006810,NaN,0.013031,0.010792,0.023793,0.008752,0.004057,0.013040,3.504720e-85,...,0.021086,0.008223,0.009299,0.016188,0.007669,0.012396,0.005554,0.018159,0.006300,0.006139
2020-12-21,0.009295,0.005137,NaN,0.015056,0.009009,0.022473,0.007850,0.016142,0.010866,1.752360e-85,...,0.020059,0.007726,0.008274,0.015156,0.003835,0.012785,0.011841,0.015957,0.004803,0.008542


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-12-19      0.017294  0.014793  0.008231  0.016740  0.018764  0.006485   
2020-12-20      0.014461  0.011875  0.006371  0.012118  0.012603  0.003907   
2020-12-21      0.012443  0.011222  0.003186  0.010709  0.008198  0.002618   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-12-19      0.009173  0.017945  0.009830  0.013102  ...  0.011914   
2020-12-20      0.006740  0.012114  0.007042  0.010072  ...  0.009627   
2020-12-21      0.005514  0.008138  0.005403  0.007756  ...  0.012661   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-12-19      0.007464  0.003762  0.005718   0.011025  0.004649  0.009761   
2020-12-20      0.007417  0.003343  0.004635   0.008473  0.004834  0.008437   
2020-12-21      0.006148  0.005628  0.004087   0.012331  0.006663  0.014812   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-12-19           -1.0  0.015022  0.003981  
2020-12-20           -1.0  0.010389  0.005376  
2020-12-21           -1.0  0.015209  0.003809  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-19,0.016311,0.005878,NaN,0.015916,0.013913,0.011190,0.015811,0.003198,0.007776,NaN,...,0.010258,0.009554,0.011921,0.014255,9.843023e-16,0.009623,0.004200,0.025860,0.017293,0.009619
2020-12-20,0.008155,0.002939,NaN,0.010100,0.014164,0.007654,0.011247,0.001599,0.006219,NaN,...,0.014441,0.006974,0.009009,0.021016,4.921512e-16,0.005565,0.002100,0.015604,0.010870,0.004809
2020-12-21,0.004078,0.001469,NaN,0.005113,0.016041,0.007399,0.006768,0.009310,0.008330,NaN,...,0.012574,0.005597,0.007102,0.010508,2.460756e-16,0.003213,0.001372,0.008245,0.006278,0.002405


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                          \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   
2020-12-19      0.007245  0.006928  0.033348  0.004059  0.026848   
2020-12-20      0.003623  0.003464  0.016674  0.002030  0.013424   
2020-12-21      0.001811  0.001732  0.008337  0.001015  0.006712   

Province_State                                                        ...  \
Admin2               Bullock    Butler   Calhoun  Chambers  Cherokee  ...   
2020-12-19      8.030943e-07  0.005775  0.016375  0.019587 -0.010417  ...   
2020-12-20      4.015472e-07  0.002887  0.008188  0.009793 -0.005208  ...   
2020-12-21      2.007736e-07  0.001444  0.004094  0.004897 -0.002604  ...   

Province_State   Wyoming                                              \
Admin2              Park        Platte  Sheridan Sublette Sweetwater   
2020-12-19      0.000020  2.789392e-07  0.019283  0.00072   0.002488   
2020-12-20      0.000010  1.394696e-07  0.009642  0.00036   0.001244   
2020-12-21      0.000005  6.973480e-08  0.004821  0.00018   0.000622   

Province_State                                                             
Admin2                 Teton     Uinta Unassigned  Washakie        Weston  
2020-12-19      7.105427e-15  0.000108  -0.501961  0.000079  1.907348e-06  
2020-12-20      3.552714e-15  0.000054  -0.501961  0.000040  9.536742e-07  
2020-12-21      1.776357e-15  0.000027  -0.501961  0.000020  4.768371e-07  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201221,AK,42563,NaN,1158414,NaN,totalTestsViral,1200977,123.0,937.0,...,1200977,0,2,06ec5bf77a86651bb51765514d8851b858b3f3a4,0,0,0,0,0,NaN
1,20201221,AL,324832,60041.0,1525510,NaN,totalTestsPeopleViral,1790301,2526.0,30994.0,...,1850342,0,770,4d2787e589a3dd11816a6bf70233d088a159da8d,0,0,0,0,0,NaN
2,20201221,AR,203107,31435.0,1777857,NaN,totalTestsViral,1949529,1078.0,10570.0,...,1980964,58,68,fb52803f1749bfcb91e5e83bcd0a0f13f79237d8,0,0,0,0,0,NaN
3,20201221,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,...,2140,0,0,92dfbe41bea321da80b8bdb60a0ee13f2d5383a8,0,0,0,0,0,NaN
4,20201221,AZ,461345,19691.0,2224376,NaN,totalTestsViral,4658586,3925.0,33195.0,...,2685721,1,158,18fe65a0b58a35c53e8a257c5387c43062382523,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

/Users/Brian/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (15,16,17,18,21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201221,AK,42563.0,NaN,1158414.0,NaN,totalTestsViral,1200977.0,123.0,937.0,...,1200977,0,2,06ec5bf77a86651bb51765514d8851b858b3f3a4,0,0,0,0,0,NaN
1,20201221,AL,324832.0,60041.0,1525510.0,NaN,totalTestsPeopleViral,1790301.0,2526.0,30994.0,...,1850342,0,770,4d2787e589a3dd11816a6bf70233d088a159da8d,0,0,0,0,0,NaN
2,20201221,AR,203107.0,31435.0,1777857.0,NaN,totalTestsViral,1949529.0,1078.0,10570.0,...,1980964,58,68,fb52803f1749bfcb91e5e83bcd0a0f13f79237d8,0,0,0,0,0,NaN
3,20201221,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,2140,0,0,92dfbe41bea321da80b8bdb60a0ee13f2d5383a8,0,0,0,0,0,NaN
4,20201221,AZ,461345.0,19691.0,2224376.0,NaN,totalTestsViral,4658586.0,3925.0,33195.0,...,2685721,1,158,18fe65a0b58a35c53e8a257c5387c43062382523,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-12-21,AK,42563,NaN,1158414,NaN,totalTestsViral,1200977,123.0,937.0,NaN,...,1200977,0,2,06ec5bf77a86651bb51765514d8851b858b3f3a4,0,0,0,0,0,NaN
2020-12-21,AL,324832,60041.0,1525510,NaN,totalTestsPeopleViral,1790301,2526.0,30994.0,NaN,...,1850342,0,770,4d2787e589a3dd11816a6bf70233d088a159da8d,0,0,0,0,0,NaN
2020-12-21,AR,203107,31435.0,1777857,NaN,totalTestsViral,1949529,1078.0,10570.0,348.0,...,1980964,58,68,fb52803f1749bfcb91e5e83bcd0a0f13f79237d8,0,0,0,0,0,NaN
2020-12-21,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,2140,0,0,92dfbe41bea321da80b8bdb60a0ee13f2d5383a8,0,0,0,0,0,NaN
2020-12-21,AZ,461345,19691.0,2224376,NaN,totalTestsViral,4658586,3925.0,33195.0,904.0,...,2685721,1,158,18fe65a0b58a35c53e8a257c5387c43062382523,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-12-21,AK,42563.0,NaN,1158414.0,NaN,totalTestsViral,1200977.0,123.0,937.0,NaN,...,1200977,0,2,06ec5bf77a86651bb51765514d8851b858b3f3a4,0,0,0,0,0,NaN
2020-12-21,AL,324832.0,60041.0,1525510.0,NaN,totalTestsPeopleViral,1790301.0,2526.0,30994.0,NaN,...,1850342,0,770,4d2787e589a3dd11816a6bf70233d088a159da8d,0,0,0,0,0,NaN
2020-12-21,AR,203107.0,31435.0,1777857.0,NaN,totalTestsViral,1949529.0,1078.0,10570.0,348.0,...,1980964,58,68,fb52803f1749bfcb91e5e83bcd0a0f13f79237d8,0,0,0,0,0,NaN
2020-12-21,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,2140,0,0,92dfbe41bea321da80b8bdb60a0ee13f2d5383a8,0,0,0,0,0,NaN
2020-12-21,AZ,461345.0,19691.0,2224376.0,NaN,totalTestsViral,4658586.0,3925.0,33195.0,904.0,...,2685721,1,158,18fe65a0b58a35c53e8a257c5387c43062382523,0,0,0,0,0,NaN
